In [368]:
# improting the basic libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
import os
%matplotlib inline
warnings.filterwarnings('ignore')


# statistical analysis of variance 
from statsmodels.formula.api import ols


# missing value remover libraries
from lightgbm import LGBMClassifier, LGBMRegressor

In [369]:
train = pd.read_csv(r'D:\House Price prediction\train.csv')
train.describe(include='all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


In [374]:
droppables = [
    col for col in train.columns 
    if train[col].isna().sum() > 0
    and (len(train) - train[col].isna().sum()) < len(train) / 3 
]
droppables
train = train.drop(columns = droppables)


In [ ]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [ ]:
class Missing_value_analysis:
    def __init__(self, df, size):
        self.df = df.copy()
        self.size = size
        
    def analysis(self):
        for i in self.df.columns:
            if self.df[i].isna().sum() > 0: 
                print(i + ' has missing values: ' + str(self.df[i].isna().sum()))
            else: 
                pass
        for i in self.df.columns:
            if self.df[i].isna().sum() < self.df.shape[0] * self.size:
                self.df.drop(columns=[i], inplace=True)
            else: pass
        
    def missing_cols(self, cols_to_impute = []):
        for i in self.df.columns:
            if self.df[i].isna().sum() >= self.df.shape[0] * self.size:
                cols_to_impute.append(i)
        
        return(list(cols_to_impute))
        
print(Missing_value_analysis(train, 0.01).analysis())
missing_cols = Missing_value_analysis(train, 0.01).missing_cols()
print(missing_cols)

LotFrontage has missing values: 259
Alley has missing values: 1369
MasVnrType has missing values: 872
MasVnrArea has missing values: 8
BsmtQual has missing values: 37
BsmtCond has missing values: 37
BsmtExposure has missing values: 38
BsmtFinType1 has missing values: 37
BsmtFinType2 has missing values: 38
Electrical has missing values: 1
FireplaceQu has missing values: 690
GarageType has missing values: 81
GarageYrBlt has missing values: 81
GarageFinish has missing values: 81
GarageQual has missing values: 81
GarageCond has missing values: 81
PoolQC has missing values: 1453
Fence has missing values: 1179
MiscFeature has missing values: 1406
None
['LotFrontage', 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [ ]:
train[missing_cols].isna().sum()

LotFrontage      259
Alley           1369
MasVnrType       872
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [ ]:
class Handling_Numerical_Missing_values:
    def __init__(self, df):
        self.df = df
        self.missing_cols = None  
        self.numeric_missing_cols = None

    def extracting_missing_cols(self):
        # Assuming Missing_value_analysis is defined elsewhere
        self.missing_cols = list(set(Missing_value_analysis(self.df, 0.01).missing_cols()))
        return self.missing_cols

    def copying(self):
        if self.missing_cols is None:
            self.extracting_missing_cols()
        data = self.df[self.missing_cols].copy()
        return data
    
    def getting_numeric_cols(self):
        if self.numeric_missing_cols is None:
            data = self.copying()
            self.numeric_missing_cols = data.select_dtypes(include=['int', 'float']).columns.tolist()
        return self.numeric_missing_cols
    
    def getting_categorical_cols(self):
        data = self.copying()
        categorical_missing_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
        return categorical_missing_cols
    
    def main(self):
        data = self.copying()
        
        
        for i in self.numeric_missing_cols:
            
            def splitting_feature(data):
                nan_col_name = f'nan_{i}'
                data[nan_col_name] = 0
                data.loc[data[i].isna(), nan_col_name] = 1
                
                trainnum = data[data[nan_col_name] == 0]
                testnum = data[data[nan_col_name] == 1]
                return nan_col_name, trainnum, testnum        
            nan_col_name, trainnum, testnum = splitting_feature(data)
            
            def preparing_feature(nan_col_name, trainnum, testnum):
                xtrain = pd.get_dummies(trainnum.drop(columns=[i, nan_col_name]))
                xtest = pd.get_dummies(testnum.drop(columns=[i, nan_col_name]))
                ytrain = trainnum[i]
                xtest = xtest.reindex(columns=xtrain.columns, fill_value=0) # this is very important, otherwise the indexes of test data will not match of the train data
                return xtrain, xtest, ytrain
            xtrain, xtest, ytrain = preparing_feature(nan_col_name, trainnum, testnum)
            
            def lgbm(xtrain, ytrain, xtest):
                lgbmr = LGBMRegressor(random_state=42, verbose = 1, n_jobs = 1)
                lgbmr.fit(xtrain, ytrain)
                ypred = lgbmr.predict(xtest)
                return ypred
            
            predicted_values = lgbm(xtrain, ytrain, xtest)
            self.df.loc[self.df[i].isna(), i] = predicted_values
# Usage
handler = Handling_Numerical_Missing_values(train)
missing_cols = handler.extracting_missing_cols()  
numeric_missing_cols = handler.getting_numeric_cols()
categorical_missing_cols = handler.getting_categorical_cols()


### Exectuting the final code for numerical missing value imputation
handler.main()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 173
[LightGBM] [Info] Number of data points in the train set: 1201, number of used features: 44
[LightGBM] [Info] Start training from score 70.049958
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 175
[LightGBM] [Info] Number of data points in the train set: 1379, number of used features: 45
[LightGBM] [Info] Start training from score 1978.506164


In [ ]:
for i in categorical_missing_cols:
    each = train[i].value_counts(normalize = True, dropna = True)
    print(each)

MiscFeature
Shed    0.907407
Gar2    0.037037
Othr    0.037037
TenC    0.018519
Name: proportion, dtype: float64
PoolQC
Gd    0.428571
Ex    0.285714
Fa    0.285714
Name: proportion, dtype: float64
Fence
MnPrv    0.558719
GdPrv    0.209964
GdWo     0.192171
MnWw     0.039146
Name: proportion, dtype: float64
BsmtQual
TA    0.456079
Gd    0.434294
Ex    0.085032
Fa    0.024596
Name: proportion, dtype: float64
GarageFinish
Unf    0.438724
RFn    0.306019
Fin    0.255257
Name: proportion, dtype: float64
GarageQual
TA    0.950689
Fa    0.034808
Gd    0.010152
Ex    0.002175
Po    0.002175
Name: proportion, dtype: float64
FireplaceQu
Gd    0.493506
TA    0.406494
Fa    0.042857
Ex    0.031169
Po    0.025974
Name: proportion, dtype: float64
Alley
Grvl    0.549451
Pave    0.450549
Name: proportion, dtype: float64
BsmtExposure
No    0.670183
Av    0.155415
Gd    0.094233
Mn    0.080169
Name: proportion, dtype: float64
BsmtFinType2
Unf    0.883263
Rec    0.037975
LwQ    0.032349
BLQ    0.023207
